In [2]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

In [3]:
# This is to load the conv part of the model in order to 
# extract feature map.
model = VGG16(weights = 'imagenet', include_top = False)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [4]:
# This is to load the whole model in order to re-train it.
model = VGG16(weights = 'imagenet', include_top = True)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [29]:
from keras.layers.core import Dense, Flatten
from keras.models import Model
base_model = VGG16(include_top = False, weights = 'imagenet', input_shape = (224,224,3))
for layer in base_model.layers:
    layer.trainable = False
model = Flatten(name='Flatten',)(base_model.output)
model = Dense(4096)(model)
model = Dense(4096)(model)
model = Dense(2, activation = 'sigmoid')(model)
final_model = Model(inputs=base_model.input, outputs = model, name = 'Modified_VGG16')

In [ ]:
from keras import optimizers
sgd = SGD(lr=0.05, decay=1e-5)
final_model.compile(loss='categorical_crossentropy',
                        optimizer=sgd, metrics=['accuracy'])

In [30]:
final_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [ ]:
# from keras.utils import multi_gpu_model

# # 将 `model` 复制到 8 个 GPU 上。
# # 假定你的机器有 8 个可用的 GPU。
# parallel_model = multi_gpu_model(model, gpus=8)
# parallel_model.compile(loss='categorical_crossentropy',
#                        optimizer='rmsprop')

# # 这个 `fit` 调用将分布在 8 个 GPU 上。
# # 由于 batch size 为 256，每个 GPU 将处理 32 个样本。
# parallel_model.fit(x, y, epochs=20, batch_size=256)